<a href="https://colab.research.google.com/github/Fat-AK/Deep-Learning-/blob/main/Copy_of_HMW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. MDPs

## 1.1 chess

 chess board :  3x3 grid with only the kings on the board. 
 The goal of the game : checkmate the opponent's king or force the opponent to resign. The player who achieves this goal is considered the winner.

State state (s): All possible board positions, players, and any additional information. In this case, there are nine possible board positions, two players (white and black), and a turn counter.

Action Space (a): All legal moves that can be made from the current board position. In this case, each player can move their king to any adjacent cell on the board, or they can choose to pass their turn.

Reward Function (r): Win +1, Loss -1, and 0 for a draw or passing the turn.

Transition Probability: The transition probability would represent the likelihood of moving from one board position to another after a move is made. In this case, the transition probability would be deterministic since there are only a limited number of possible moves from each board position.

Discount Factor: The discount factor is a value between 0 and 1 that represents the importance of future rewards compared to immediate rewards. In chess, a discount factor of 1 can be used because the outcome of the game is determined at the end.

## 1.2 LunarLander

The link does not work 

## 1.3 Model Based RL: Accessing Environment Dynamics

### Discuss the Policy Evaluation and Policy Iteration algorithms from the lecture.They explicitly make use of the environment dynamics (p(s′, r|s, a)).

Policy evaluation is the process of calculating the value function for a given policy (π) in an MDP. The value function determines the expected return (Gt) for an agent starting in a particular state (s) and following a specific policy. The policy evaluation algorithm estimates the value function iteratively, starting from an arbitrary initial value function and updating it until it converges to the true value function.

1- Initialize the value function arbitrarily
Repeat:
    For each state s:
    Update the value function V(s) = Σ p(s', r|s, a) [r + γV(s')]
    Where p(s', r|s, a) is the probability of transitioning to state s' and receiving reward r when taking action a from state s, and γ is the discount factor.
    
Policy Iteration:

Policy iteration is an RL algorithm that alternates between policy evaluation and policy improvement to find the optimal policy for an MDP. The algorithm starts with an arbitrary policy and iteratively updates it until the optimal policy is found.

The key step in improving the policy was understanding we could compute the optimal action given the current policy and state value by looking at the expected return of taking any action a in a given state and afterwards following the current policy:
∑s′,r[p(s′,r|s,a)(r+Vπ(s′))]

if we compare the expected Return of all possible different actions in a given state, we can just pick the best one to optimize the policy:
π(s)=argmaxa[∑s′,r[p(s′,r|s,a)(r+Vπ(s′))]]


### Explain what the environment dynamics (i.e. reward function and state transition function) are and give at least two examples.

The environment dynamics in a reinforcement learning problem refer to the reward function and state transition function, which determine the consequences of an agent's actions in the environment.

Reward Function: The reward function maps a state and action pair to a numerical reward value that represents how good or bad that action was for the agent. The goal of the agent is to learn a policy that maximizes the cumulative reward it receives over time. A well-designed reward function is critical for guiding the agent towards desirable behavior.

Example 1: Consider a self-driving car navigating through traffic. A reward function might assign positive rewards for reaching the destination safely and quickly, negative rewards for causing accidents or violating traffic laws, and intermediate rewards for following a smooth and comfortable driving style. The reward function should be designed to prioritize safety, as that is the most important objective.

Example 2: In a game of chess, the reward function might assign a positive reward for winning the game, a negative reward for losing, and a zero reward for a draw. The reward function could also assign intermediate rewards for taking control of the center of the board, putting the opponent's king in check, or capturing the opponent's pieces.

State Transition Function: The state transition function maps a current state and action pair to a next state. It determines how the environment evolves over time in response to the agent's actions. In many reinforcement learning problems, the state transition function is stochastic, meaning that the next state is not entirely predictable and can depend on random factors.



### Discuss: Are the environment dynamics generally known and can practically be used to solve a problem with RL?


#  2.Implementing a GridWorld

### Look up some examples of GridWorld! List at least three links or references to different GridWorlds you could find online

1- https://cs.stanford.edu/people/karpathy/reinforcejs/gridworld_dp.html
